## Web Scraping : EN-TH, ES-EN POS & translate 

In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [ ]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [ ]:
def get_pos(word):
    url = "https://dictionary.cambridge.org/dictionary/english-thai/"+word
    headers = requests.utils.default_headers()

    headers.update(
        {
            'User-Agent': 'My User Agent 1.0',
        }
    )

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')
    # get pos
    pos = []
    meaning = []
    try:
        ########### English ###################
        html_tables = soup.find_all("span", {"class": 'di-info'})
        pos =[data.find("span", {"class": 'pos dpos'}).get_text() for data in html_tables]

        ########### Espanol ###################
        # pos = soup.find("span", {"class": 'pos dpos'}).get_text()
    except:
        pass
    # get meaning
    try:
        ########### English ###################
        html= soup.find_all("div", {"class": 'di-body normal-entry-body'})
        meaning  =[l.find("span",{"class":"trans dtrans"}).getText() for l in html]

        ########### Espanol ###################
        # meaning = soup.find("span",{"class":"trans dtrans"}).get_text()
    except:
        pass
    return pos,meaning

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
file_name = list(uploaded.keys())[0]

In [ ]:
df = pd.read_excel(file_name)

In [ ]:
df.head()

,word,verb,noun,adjective,other1,other2
0,Paradise,NaN,NaN,NaN,NaN,NaN
1,HE,NaN,NaN,NaN,NaN,NaN
2,God,NaN,NaN,NaN,NaN,NaN
3,Happiness,NaN,NaN,NaN,NaN,NaN
4,Love,NaN,NaN,NaN,NaN,NaN


In [ ]:
nouns = ["noun","noun plural","noun singular","pronoun"]
pos_head = ["verb","adjective"]
for i in range (len(df)):
    print(i,str(df["word"][i]))
    pos, meaning = get_pos(str(df["word"][i]))
    if len(pos)==0 or len(pos) != len(meaning) : 
        pass
    for j in range (len(pos)):
        if pos[j] in pos_head:
                df[pos[j]][i] = meaning[j]
        elif pos[j] in nouns:
                df["noun"][i] = meaning[j]
        elif pd.isna(df["other1"][i]):
                df["other1"][i] = meaning[j] + " (" + pos[j] + ")"
        else:
                df["other2"][i] = meaning[j] + " (" + pos[j] + ")"

In [ ]:
df

,word,verb,noun,adjective,other1,other2
0,Paradise,NaN,สถานที่หรือสภาวะแห่งความสุข,NaN,NaN,NaN
1,HE,NaN,ผู้ชาย; สัตว์เพศผู้,NaN,เขา (ผู้ชาย) (prefix),NaN
2,God,NaN,พระเจ้า,NaN,NaN,NaN
3,Happiness,NaN,ความสุข,NaN,NaN,NaN
4,Love,ชอบ,ความรัก,NaN,NaN,NaN
...,...,...,...,...,...,...
10010,pandemic,NaN,NaN,NaN,NaN,NaN
10011,sanitizer,NaN,NaN,NaN,NaN,NaN
10012,self-quarantine,NaN,NaN,NaN,NaN,NaN
10013,ventilator,NaN,เครื่องระบายอากาศ,NaN,NaN,NaN


In [ ]:
df = df.fillna("-")

In [ ]:
df.to_excel("output.xlsx")  
files.download('output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Web Scraping : EN-EN POS 


In [ ]:
def get_pos(word):
    url = "https://dictionary.cambridge.org/dictionary/english/"+word
    headers = requests.utils.default_headers()

    headers.update(
        {
            'User-Agent': 'My User Agent 1.0',
        }
    )

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')
    try:
      html_tables = soup.find_all("div", {"class": 'pos-header dpos-h'})
      pos = set()
      for parts in html_tables:
        parts = parts.find_all("span", {"class": 'pos dpos'})
        for part in parts:
          pos.add(part.get_text())
    except:
        pass
    return pos

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name)
df.head()

In [ ]:
nouns = ["noun","noun plural","noun singular","pronoun"]
pos_head = ["verb","adjective"]
for i in range (len(df)):
    print(i,str(df["word"][i]))
    pos = get_pos(str(df["word"][i]))
    if len(pos)==0 : 
        pass
    for p in pos:
        if p in pos_head:
                df[p][i] = df["word"][i]
        elif p in nouns:
                df["noun"][i] = df["word"][i]
        elif pd.isna(df["other1"][i]):
                df["other1"][i] = df["word"][i] + " (" + p + ")"
        else:
                df["other2"][i] = df["word"][i] + " (" + p + ")"

In [ ]:
df = df.fillna("-")
df

In [ ]:
df.to_excel("output.xlsx")  
files.download('output.xlsx')